# FYSC12: Beta decay and Fermi Kurie plot
## Data analysis code and lab notebook

### Welcome to your Jupyter Notebook for the FYSC12 beta lab!
In here, you will find all the code needed to measure the $Q$ value of the beta decay of $^{90}$Sr from the data taken during the lab.

Please treat this as your "lab notebook": by keeping notes on experimental settings etc inside this page, you should have a complete document on *your* experiment by the end of the day.

## Warm-up Questions
*These questions need to be answered _before_ coming to the lab*

* How does the electron energy spectrum look like for a $\beta^-$ decay?
  Why?

_Answer_:

* What is the Fermi-Kurie plot and how does it relate to the $Q$ value of
  the process?

_Answer_:

* Why is $^{90}Sr$ a suitable sample to use in this laboratory exercise?
  What might be a challenge using this isotope?

_Answer_:

* Calculate the Q-value for the process $^{90}Sr\rightarrow ^{90}Y + e^- +
  \bar{\nu}$. Hint: Some terms in the equation can be neglected, why? 

_Answer_:

* By what mechanisms does a photon/charged particle lose energy
  when traversing material?

_Answer_:

* Why is plastic a good material for a scintillator? What
  properties can you think of that might be important to consider?

_Answer_:

* What is the function of the photomultiplier tube? What properties are important?

_Answer_:

### Some basic things on Python:

In [ ]:
## assign a variable:
myvar = 23.5
## print a variable to the notebook:
print(myvar)
## print text + variable:
print('this is myvar: {}'.format(myvar))
## help with any function:
help(print)

For data storage and analysis, we rely on the *numpy* and *scipy* packages. And to plot things, we use the *matplotlib* library. We need to import both before the first use and initialize the interactive plots inside the notebook:

In [ ]:
import numpy as np
import scipy
import matplotlib
# choose a backend for web applications; remove for stand-alone applications:
matplotlib.use('Agg')
import matplotlib.pyplot as plt
# enable interactive notebook plots (
# alternative: use 'inline' instead of 'notebook' for static images)
%matplotlib notebook

### Let's take some data!
#### describe your experiment and settings here:

.......

**Important**: write down any settings you apply! And take pictures of the setup/sources.

#### Load the data from the MCA
We have a little helper library called MCA.py that we load here. It provides a new class *Spectrum* and a routine to load (ASCII) files from the MCA: *load_spectrum("filename")*

In [ ]:
import MCA
sr90 = MCA.load_spectrum(filename =  'beta_lab_calib/sr90.Spe')
sr90.name = "Sr-90"

Now it's time to plot the data we just measured!
Run the cell below to see the result appear inside the notebook:

### Plot the first data

In [ ]:
fig, axes = plt.subplots() # makes new plot
axes.plot(sr90.x, sr90.y, linestyle="steps")

## Could be useful to see this in log scale..?
#plt.yscale('log')

The result looks (hopefully) more-or-less identical to what we have seen in the MCA software. Good, let's proceed with the analysis then!

**Note**: if you use a pure Python program (not a notebook like we do today) and want to plot with matplotlib, then you need to adjust the indicated lines around the matplotlib import statement add the *plt.show()* command (without arguments) to actually _show_ the plots in a separate window.

#### Questions:
- what is the Q value (in channels) that you read off the plot above? How confident (in units of channels) are you with that reading?

**Answer here**:

## Energy calibration of the detector
We have two different sources available: Cs-137 and Bi-207

Your task:
- have your lab supervisor hand you a source and instruct you to place it at the measurement location
- take a spectrum
- while waiting for the measurement: study the incoming MCA spectrum (alternatively, plot the already recorded data here)
- research at what intensities and energies the source emitts internal conversion electrons (electron capture process). *Hint:* The Nudat database might be useful: http://www.nndc.bnl.gov/nudat2
- identify what emissions contribute to the peaks you see in the spectrum
- our detector cannot resolve small differences in energy, so the peaks we see might stem from several emissions at different energies; calculate the mean energy taking into account the (relative) intensities

When you are ready, plot the new raw data:

In [ ]:
calsource1_Epeak = []
calsource1_Epeak.append(123) # TODO: put in right energy!
print ("Mean E peak #1: {} MeV".format(calsource1_Epeak[0]))

In [ ]:
calsource1 = MCA.load_spectrum(filename = 'some_file.mca') # TODO: put right file
calsource1.name = "Put isotope name here" # TODO: put correct name 

fig, axes = plt.subplots() # makes new plot
plt.plot(calsource1.x, calsource1.y, linestyle="steps", label = calsource1.name)

## Let's make this plot a little nicer!

plt.xlabel('channel number')
plt.ylabel('counts')
plt.title(calsource1.name)
plt.legend()


### Determining the IC peak position and width via Gaussian fits

- what is the advantage of using Gaussian fits over reading off the value by eye?

In [ ]:
import fithelpers
gfits = fithelpers.fit_all_gaussians(calsource1.x, calsource1.y, loglevel="WARNING")

In [ ]:
fig, axes = plt.subplots() # makes new plot

axes.plot(calsource1.x, calsource1.y, label = calsource1.name, linestyle="steps")
for g in gfits:
    ## plot the gaussian fit
    axes.plot(calsource1.x, g.value(calsource1.x), 
              label = "Gaussian fit, sgm="+str(round(g.sigma)))
plt.title(calsource1.name)


In [ ]:
## set up a new plot
fig, axes = plt.subplots() # makes new plot
plt.grid(True)
plt.xlabel('channel number')
plt.ylabel('counts')
plt.title(calsource1.name)

## plot the cs-137 data
plt.plot(calsource1.x, calsource1.y, linestyle='steps',label=calsource1.name)
## loop over all the fit results and enumerate them with an index
for idx, g in enumerate(gfits):
    ## print a line with the fit result
    print("Fitted Gaussian #{}: {}".format(idx, g.as_string()))
    ## plot this gaussian fit too
    plt.plot(calsource1.x, g.value(calsource1.x), 
             label="Gauss fit {}".format(idx))
    ## add a label to the peak position with the fit index
    plt.annotate(idx,
            xy=(g.mu, g.A), xycoords='data', # where to point to
            xytext=(15, 15), textcoords='offset points', # offset
            arrowprops=dict(arrowstyle="->")) # arrowstyle
    
plt.legend() # make legend visible


### Identify the peak and associate an energy to it

In [ ]:
calsource1_channel = [ gfits[0].mu ]
calsource1_sigma   = [ gfits[0].sigma ]


### Now repeat the above steps for the 2nd calibration source
- start the data taking and study the incoming MCA spectrum 
- research at what intensities and energies the source emitts internal conversion electrons 
- identify what emissions contribute to the peaks we see
- calculate the mean energy taking into account the (relative) intensities

In [ ]:
calsource2_Epeak = []
calsource2_Epeak.append(123)
for idx, peak in enumerate(calsource2_Epeak):
    print ("Mean E peak #{}".format(idx) + ": {} MeV".format(peak))


In [ ]:
calsource2 = MCA.load_spectrum(filename = 'file_name_here.mca') # TODO: add file!

if calsource2:
    calsource2.name = "other isotope name" # TODO: add isotope's name here!
    gfits2 = fithelpers.fit_all_gaussians(calsource2.x, calsource2.y, loglevel="WARNING")
    fig, axes = plt.subplots() # makes new plot
    plt.plot(calsource2.x, calsource2.y, linestyle="steps", label = calsource2.name)

    for idx, g in enumerate(gfits2):
        print("Fitted Gaussian #{}: {}".format(idx, g.as_string()))
        plt.plot(calsource2.x, g.value(calsource2.x), 
                 label="Gauss fit {}".format(idx))
        plt.annotate(idx,
                xy=(g.mu, g.A), xycoords='data', # where to point to
                xytext=(15, 15), textcoords='offset points', # offset
                arrowprops=dict(arrowstyle="->")) # arrowstyle
    
    ## Let's make this plot a little nicer!
    plt.xlabel('channel number')
    plt.ylabel('counts')
    plt.yscale('log')
    plt.ylim(ymin=1)
    plt.title(calsource2.name)
    plt.legend()

Let's try to fit even the third (and weakest) peak! Try to make it visible in the plot above and perform a fit at the position you identified by eye using *fithelpers.fit_gaussian_at_pos()*:

In [ ]:
if calsource2:
    g3rd = fithelpers.fit_gaussian_at_pos(calsource2.x, calsource2.y, pos=661, npoints=30)
    plt.plot(calsource2.x, g3rd.value(calsource2.x), label="3rd peak")

#### Now assemble above results into data structures we can plot later

In [ ]:
if calsource2:
    calsource2_channel = [ gfits2[0].mu, gfits2[1].mu, g3rd.mu ]
    calsource2_sigma   = [ gfits[0].sigma, gfits2[1].sigma, g3rd.sigma ]

### Plot the energy calibration figure

In [ ]:
fig, axes = plt.subplots() # makes new plot
plt.grid(True)
plt.xlabel('channel number')
plt.ylabel('energy [MeV]')
plt.title("Energy Calibration")
## PLOT the energy calibration data including uncertainties
plt.errorbar(x=calsource1_channel, 
             y=calsource1_Epeak, 
             xerr=calsource1_sigma, 
             marker='o',
             label=calsource1.name)

# if we have two calibration sources, plot the second one here:
if calsource2:
    plt.errorbar(x=calsource2_channel, 
                 y=calsource2_Epeak, 
                 xerr=calsource2_sigma, 
                 marker='o',
                 label=calsource2.name)

## might want to COMBINE data arrays from different calibration sources for the fit:
## use
## new_list = list1 + list2
## to do so. Then change the data set in the fit command.

## linear regression of the data
## http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html
## or "help(scipy.stats.linregress)"
slope = 1.
intercept = 0.
# .... something is missing here....

## ADVANCED METHOD TO FIT:
## use "curve_fit" which allows to take uncertainties into account!
## http://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html
## or "help(scipy.optimize.curve_fit)"

## HINT 1: curve_fit assumes sigma is on the y values -- need to "invert" fit
## HINT 2: need to calculate the error on the parameters from the covariance matrix

print("Determined calibration constants from linear regression: E [MeV] = "+str(slope)+"*N_ch + " + str(intercept))
x = np.arange(1,512)
plt.plot(x,slope*x+intercept,label="linear regression")
plt.legend()

## apply energy calibration
print("Reloading {} spectrum".format(sr90.name))
sr90 = MCA.load_spectrum(filename =  sr90.filename)
print("Applying calibration constants")
sr90.calibrate(slope,intercept)

fig, axes = plt.subplots() # makes new plot
plt.grid(True)
plt.ylabel('counts')
plt.xlabel('energy [MeV]')
plt.title("Sr-90 spectrum")
plt.yscale('log')
plt.plot(sr90.x,sr90.y,label="Sr-90")


### Energy calibration, optional steps:
- can you demonstrate that the peaks we have identified in fact stem from electrons? How?
- improve the calibration by taking into account all IC emissions from Bi-207 and Cs-137
- improve the fit by taking into account uncertainties!
- how can you evaluate the quality of the fit? Look into the documentation (or code) of the routine and interpret what information it returns!

## Fermi-Kurie Plot
Let's calculate $Q-T_e$ and look at the resulting distribution!

*You are lucky, all the formulars have already been typed in -- and look how easy Python makes the calculations!*

In [ ]:
## fermi-kurie calculations:
mec2 = 0.510998910 ## e mass in MeV
pc = np.sqrt((sr90.x + mec2)**2 - mec2**2)
A = pc/mec2
f = 1.3604*A*A + 0.1973*A + 0.0439
Ee = (sr90.x + mec2)
QminTe = np.sqrt((sr90.y*pc)/(Ee*f))

## not plot the calculated data
fig, axes = plt.subplots() # makes new plot
plt.grid(True)
plt.xlabel('Te [MeV]')
plt.ylabel('Q-Te')
plt.title("{} Fermi-Kurie".format(sr90.name))
plt.plot(sr90.x, QminTe, label="data", linestyle="steps")
    
## linear regression of the F-K plot see
## http://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.stats.linregress.html
## the fit does not really work on the edges of the F-K plot, so we
## take the linear region of lower_limit<E [MeV]<upper_limit (to
## be determined from the plot)
lower_limit, upper_limit = 1,11 ## initialize
try:
    ## search for the bins that match our criteria
    ## first elements ([0][0]) indicate first bin matching our criteria
    lower_limit = np.where(sr90.x>0.5)[0][0]
    upper_limit = np.where(sr90.x>2.)[0][0]
except IndexError:
    print("Could not find any bins to fit! Please check the limit settings!")

slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(
    sr90.x[lower_limit:upper_limit], QminTe[lower_limit:upper_limit])

x = np.arange(0,2.5,0.05) ## generate x axis for fit result (start, stop, stepsize)
plt.plot(x,slope*x+intercept,label="linear regression")

## now the Q value is determined by where the linear regression 
## intersects with the x axis (Q-Te = 0)
Q = -intercept/slope

## print results
print("Determined linear regression to Fermi-Kurie plot: Q-Te = " 
          + str(slope)+"*Te + " + str(intercept))
print("===> Q value: Q = "+str(Q)+" MeV ")
## label plot with Q value
plt.annotate(r'$Q_{Y-90}$ = '+"{:.3f}".format(Q)+' MeV', # text to put there
            xy=(Q, 0),                         # where to point to
            xytext=(0., 60),                   # offset for text
            textcoords='offset points',
            arrowprops=dict(arrowstyle="->", color = 'red'))

## try to fit the Sr-90 beta+ decay by subtracting Y-90 beta+ first
QminTe -= sr90.x*slope+intercept
plt.plot(sr90.x, QminTe, label="data, Y-90b+ subtracted", linestyle="steps")
## fit the Sr-90b+:
lower_limit = np.where(sr90.x>0.3)[0][0]
upper_limit = np.where(sr90.x>0.7)[0][0]
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(
    sr90.x[lower_limit:upper_limit], QminTe[lower_limit:upper_limit])
plt.plot(x,slope*x+intercept,label="linear regression")

Q = -intercept/slope
print("Determined linear regression to Fermi-Kurie plot: Q-Te = " 
          + str(slope)+"*Te + " + str(intercept))
print("===> Q Sr-90 value: Q = "+str(Q)+" MeV ")
plt.annotate(r'$Q_{Sr-90}$ = '+"{:.3f}".format(Q)+' MeV', # text to put there
            xy=(Q, 0),                         # where to point to
            xytext=(0., 20),                   # offset for text
            textcoords='offset points',
            arrowprops=dict(arrowstyle="->", color = 'red'))

plt.legend()

# Discussion

At the end of the lab, it is time to present your findings and results to your colleagues! Export any plots or pictures you want to show and explain what you have learned.

# Conclusions

The following questions should be discussed and answered during the final discussion round:
* What have we learned more since the measurements this morning?
* Were any of the results particularly surprising for you?
* Are we now ready to publish the results? What would we have to
  do further?
* What are the dominant uncertainties you are aware of? What causes them?
* With all we know now, what would you do to improve the
  experiment? How would you remove limiting factors in particular to its precision?